# Benchmark modeli CNN na ImageNet v1

## Instalacja bibliotek

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

!pip install -q pytorch-lightning torchvision codecarbon scikit-learn pillow torchprofile

## Pobranie danych ImageNet v1

In [ ]:
from google.colab import files  
files.upload() # Służy do umieszczenia pliku kaggle.json

!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download sautkin/imagenet1kvalid
!unzip /content/imagenet1kvalid.zip -d /content/imagenet1kv1_valid

## Importy i konfiguracja

In [ ]:
import os
import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.models import (
    efficientnet_v2_s, EfficientNet_V2_S_Weights,
    resnet50, ResNet50_Weights,
    densenet201, DenseNet201_Weights,
    convnext_tiny, ConvNeXt_Tiny_Weights,
    mobilenet_v3_large, MobileNet_V3_Large_Weights
)
from codecarbon import EmissionsTracker
from sklearn.metrics import f1_score, log_loss, roc_auc_score, accuracy_score
from torchprofile import profile_macs
import numpy as np
import pandas as pd
from IPython.display import display

imagenet_path = "/content/imagenet1kv1_valid"
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_sizes = [128]

## Konfiguracja modeli

In [ ]:
MODEL_CONFIGS = {
    "efficientnet_v2_s":  (efficientnet_v2_s,   EfficientNet_V2_S_Weights.IMAGENET1K_V1),
    "resnet50":           (resnet50,            ResNet50_Weights.IMAGENET1K_V1),
    "densenet201":        (densenet201,         DenseNet201_Weights.IMAGENET1K_V1),
    "convnext_tiny":      (convnext_tiny,       ConvNeXt_Tiny_Weights.IMAGENET1K_V1),
    "mobilenet_v3_large": (mobilenet_v3_large,  MobileNet_V3_Large_Weights.IMAGENET1K_V1),
}

## Funkcja przygotowująca DataLoader

In [ ]:
def get_dataloader(model_weights, batch_size):
    transform = model_weights.transforms()
    ds = datasets.ImageFolder(imagenet_path, transform=transform)
    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=min(os.cpu_count() // 2, batch_size),
        pin_memory=True,
        persistent_workers=False,
        prefetch_factor=2
    )

## Benchmark modeli

In [ ]:
results = []

for model_name, (model_fn, weights) in MODEL_CONFIGS.items():
    for bs in batch_sizes:
        model = model_fn(weights=weights).to(device).eval()
        num_params = sum(p.numel() for p in model.parameters())

        example = torch.randn(1, 3, 224, 224).to(device)
        with torch.no_grad():
            macs = profile_macs(model, example)

        loader = get_dataloader(weights, bs)

        if os.path.exists("/tmp/.codecarbon.lock"):
            os.remove("/tmp/.codecarbon.lock")
        tracker = EmissionsTracker(project_name=f"{model_name}_bs{bs}", log_level="error")
        tracker.start()

        if device == "cuda":
            torch.cuda.reset_peak_memory_stats()
        t0 = time.time()

        all_probs = []
        all_preds = []
        all_labels = []
        n_images = 0

        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                probs = F.softmax(out, dim=1)
                preds = probs.argmax(dim=1)

                all_probs.append(probs.cpu().numpy())
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())
                n_images += x.size(0)

        duration = time.time() - t0
        peak_mem = torch.cuda.max_memory_allocated() / 1024**2 if device == "cuda" else 0
        co2_kg = tracker.stop()

        probs = np.vstack(all_probs)
        preds = np.hstack(all_preds)
        labels = np.hstack(all_labels)

        f1 = f1_score(labels, preds, average="macro", zero_division=0)
        acc = accuracy_score(labels, preds)
        ll = log_loss(labels, probs, labels=list(range(1000)))
        try:
            auroc = roc_auc_score(labels, probs, multi_class="ovr")
        except:
            auroc = float("nan")

        throughput = n_images / duration
        latency_ms_img = (duration / n_images) * 1000

        results.append({
            "model":               model_name,
            "num_params":          num_params,
            "macs_million":        macs,
            "accuracy":            acc,
            "f1_macro":            f1,
            "auroc":               auroc,
            "logloss":             ll,
            "time_s":              duration,
            "throughput_img_s":    throughput,
            "latency_ms_img":      latency_ms_img,
            "peak_mem_mib":        peak_mem,
            "co2_kg":              co2_kg,
        })

## Wyświetlenie wyników

In [ ]:
df = pd.DataFrame(results)
display(df)